In [52]:
%pip install --quiet kaggle pandas dlt[bigquery] google-cloud-storage gcsfs google-cloud-bigquery google-cloud-bigquery-storage

Note: you may need to restart the kernel to use updated packages.


In [53]:
import os, zipfile, glob, shutil
import pandas as pd
import dlt
from google.cloud import storage

In [54]:
# Create the .kaggle directory if it doesn't exist
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

# Copy your Kaggle credentials to the default location
shutil.copy("c:/Users/ahmed/Desktop/de-healthcare-pipeline/dlt-gcp-ingestion/kaggle/kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))

# Ensure the file has the correct permissions
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

print("Kaggle credentials set up successfully.")

Kaggle credentials set up successfully.


In [55]:
import os

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "c:/Users/ahmed/Desktop/de-healthcare-pipeline/secretkeys/credentials.json"

print("GCP credentials set up successfully.")

GCP credentials set up successfully.


In [56]:
# 3. Download from Kaggle into a tmp folder
os.makedirs("tmp", exist_ok=True)
os.system("kaggle datasets download -d meirnizri/covid19-dataset -p tmp --unzip")

0

In [57]:
# 4. Find the CSV in tmp/
csv_path = glob.glob("tmp/covid data.csv")[0]
print("✅ Downloaded CSV:", csv_path)

✅ Downloaded CSV: tmp/covid data.csv


In [58]:
import os
# This environment variable can be used to change the BigQuery location between US/EU as needed.

os.environ["DESTINATION__BIGQUERY__LOCATION"] = "EU"

print("✅ BigQuery destination location set to EU successfully.")

✅ BigQuery destination location set to EU successfully.


In [59]:
from google.cloud import storage
import os

# Set up GCS client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "c:/Users/ahmed/Desktop/de-healthcare-pipeline/secretkeys/credentials.json"
bucket_name = "covidepidemetrics-bucket"  # Replace with your GCS bucket name
csv_path = "tmp/covid data.csv"  # Path to your CSV file
gcs_path = "raw/covid19_data.csv"  # Path in GCS

# Upload CSV to GCS
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(gcs_path)
blob.upload_from_filename(csv_path)

print(f"✅ Uploaded to GCS: gs://{bucket_name}/{gcs_path}")

✅ Uploaded to GCS: gs://covidepidemetrics-bucket/raw/covid19_data.csv


In [60]:
import dlt
import pandas as pd

@dlt.resource
def covid_data_from_gcs():
    # Read the CSV file directly from GCS
    df = pd.read_csv(f"gs://{bucket_name}/{gcs_path}")
    for row in df.itertuples(index=False, name=None):
        yield dict(zip(df.columns, row))

In [61]:
# 7. Create & run the DLT pipeline into your BigQuery dataset
pipeline = dlt.pipeline(
    pipeline_name="covid_data_pipeline",
    destination="bigquery",
    dataset_name="covid19_dataset",  # BigQuery dataset name
    dev_mode=False  # Disable dev_mode for production-like behavior
)

# Run the pipeline
load_info = pipeline.run(covid_data_from_gcs())
print("✅ Data loaded to BigQuery.", load_info)

✅ Data loaded to BigQuery. Pipeline covid_data_pipeline load step completed in 23.56 seconds
1 load package(s) were loaded to destination bigquery and into dataset covid19_dataset
The bigquery destination used None@covidepidemetrics location to store data
Load package 1744988330.987893 is LOADED and contains no failed jobs
